In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [ ]:
np.random.seed(500)

In [ ]:
Corpus = pd.read_csv("/content/tagged02-07-2022-122843.csv",encoding='utf8')

In [ ]:
Corpus

,order,index,ENCNTR_ID,PERSON_ID,dcNoteFinal,Tag
0,0,0,63659760,88639569,"* Patient: BOARD, CATHERINE ...",1
1,1,1,63927816,78432102,"b""EMORY UNIVERSITY HOSPITAL MIDTOWN SUMMARY NA...",0
2,2,2,64686569,89329608,"b""EMORY UNIVERSITY HOSPITAL SUMMARY NAME: ...",0
3,3,3,65001068,69612068,"b""* TX_RTF32 10.1.323.501* d469a1f3* EMORY UNI...",0
4,4,4,65330296,90343247,EMORY UNIVERSITY HOSPITAL SUMMARY NAME: ...,2
...,...,...,...,...,...,...
146,146,303,66866343,19337092,"* Patient: WILMONT, SELENA R ...",1
147,147,304,66767408,51120121,"b""* Patient: MOORE, LUTHER LAVELLE ...",1
148,148,310,66940878,90116249,"b""* TX_RTF32 10.1.323.501* d469a1f3* EMORY UNI...",1
149,149,311,66738084,18912196,"b""EMORY UNIVERSITY HOSPITAL MIDTOWN SUMMARY NA...",1


In [ ]:
Corpus['Tag'] = [int(str(entry).replace('2','0')) for entry in Corpus['Tag']]

In [ ]:
Corpus['Tag'] = [int(str(entry).replace('4','0')) for entry in Corpus['Tag']]

In [ ]:
Corpus['Tag'] = [int(str(entry).replace('3','1')) for entry in Corpus['Tag']]

In [ ]:
Corpus

,order,index,ENCNTR_ID,PERSON_ID,dcNoteFinal,Tag
0,0,0,63659760,88639569,"* Patient: BOARD, CATHERINE ...",1
1,1,1,63927816,78432102,"b""EMORY UNIVERSITY HOSPITAL MIDTOWN SUMMARY NA...",0
2,2,2,64686569,89329608,"b""EMORY UNIVERSITY HOSPITAL SUMMARY NAME: ...",0
3,3,3,65001068,69612068,"b""* TX_RTF32 10.1.323.501* d469a1f3* EMORY UNI...",0
4,4,4,65330296,90343247,EMORY UNIVERSITY HOSPITAL SUMMARY NAME: ...,0
...,...,...,...,...,...,...
146,146,303,66866343,19337092,"* Patient: WILMONT, SELENA R ...",1
147,147,304,66767408,51120121,"b""* Patient: MOORE, LUTHER LAVELLE ...",1
148,148,310,66940878,90116249,"b""* TX_RTF32 10.1.323.501* d469a1f3* EMORY UNI...",1
149,149,311,66738084,18912196,"b""EMORY UNIVERSITY HOSPITAL MIDTOWN SUMMARY NA...",1


In [ ]:
print(Corpus['dcNoteFinal'])

0      *   Patient:    BOARD, CATHERINE              ...
1      b"EMORY UNIVERSITY HOSPITAL MIDTOWN SUMMARY NA...
2      b"EMORY UNIVERSITY HOSPITAL SUMMARY NAME:     ...
3      b"* TX_RTF32 10.1.323.501* d469a1f3* EMORY UNI...
4      EMORY UNIVERSITY HOSPITAL SUMMARY NAME:       ...
                             ...                        
146    *   Patient:    WILMONT, SELENA R             ...
147    b"*   Patient:    MOORE, LUTHER LAVELLE       ...
148    b"* TX_RTF32 10.1.323.501* d469a1f3* EMORY UNI...
149    b"EMORY UNIVERSITY HOSPITAL MIDTOWN SUMMARY NA...
150    *   Patient:    MYERS, PHILIP              MRN...
Name: dcNoteFinal, Length: 151, dtype: object


In [ ]:
# Step - a : Remove blank rows if any.
Corpus['dcNoteFinal'].dropna(inplace=True)

In [ ]:
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['dcNoteFinal'] = [entry.lower().replace('b"','') for entry in Corpus['dcNoteFinal']]

In [ ]:
print(Corpus['dcNoteFinal'])

0      *   patient:    board, catherine              ...
1      emory university hospital midtown summary name...
2      emory university hospital summary name:       ...
3      * tx_rtf32 10.1.323.501* d469a1f3* emory unive...
4      emory university hospital summary name:       ...
                             ...                        
146    *   patient:    wilmont, selena r             ...
147    *   patient:    moore, luther lavelle         ...
148    * tx_rtf32 10.1.323.501* d469a1f3* emory unive...
149    emory university hospital midtown summary name...
150    *   patient:    myers, philip              mrn...
Name: dcNoteFinal, Length: 151, dtype: object


In [ ]:
print(Corpus['dcNoteFinal'][0])

*   patient:    board, catherine              mrn: euh_02058777              fin: 39093702276               age:    80 years      sex:   female      dob:   5/25/1932   associated diagnoses:    none   author:    sadeghi yarandi, shadi     discharge information   discharge summary information :         date of admission: 10/2/2012 .        date of discharge: 10/10/2012 .        reason for admission:  acute decompensated right side heart failure  .        discharge diagnosis:  congestive heart failure exacerbation severely depressed rv function. lvef 55%. atrial fibrillation s/p pacemaker. currently paced. rate controlled. copd diabetes mellituslast hgba!c 6.2% history of coronary artery disease chronic kidney disease stage iii gout   .     results review   radiology results     echocardiogram 8/2012:    1. left ventricular ejection fraction is 55%.  2. severely dilated left atrium.  3. severely dilated right atrium.  4. trivial pericardial effusion as described below.  5. mild tricuspid 

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Step - b2: remove punctuation

from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
Corpus['dcNoteFinal'] = [tokenizer.tokenize(entry) for entry in Corpus['dcNoteFinal']]
#tokenizer.tokenize('Eighty-seven miles to go, yet.  Onward!')


In [ ]:
print(Corpus['dcNoteFinal'][1])

['emory', 'university', 'hospital', 'midtown', 'summary', 'name', 'logan', 'la', 'shawn', 'nicole', '3042739', 'no', '36004532284', 'date', '10', '11', '2012', 'date', '10', '14', '2012', 'no', '058570', 'md', 'imran', 'baloch', 'm', 'd', 'nephrologist', 'dr', 'jason', 'cobb', 'care', 'physician', 'dr', 'george', 'jones', 'diagnoses', 'chest', 'pain', 'workup', 'negative', 'with', 'a', 'vq', 'scan', 'hypertensive', 'urgency', 'now', 'blood', 'pressure', 'in', 'much', 'better', 'control', 'chronic', 'kidney', 'disease', 'progressing', 'to', 'stage', '5', 'human', 'immunodeficiency', 'virus', 'disease', 'history', 'of', 'chronic', 'back', 'pain', 'history', 'of', 'dilated', 'cardiomyopathy', 'with', 'last', 'ejection', 'fraction', 'being', '55', 'in', '07', '2012', 'which', 'is', 'an', 'improvement', 'from', 'an', 'echocardiogram', '02', '2012', 'when', 'it', 'was', '30', 'to', '35', 'medications', 'ziagen', '300', 'mg', '2', 'tabs', 'by', 'mouth', 'every', 'day', 'aspirin', '81', 'mg', 

In [ ]:
print(Corpus['dcNoteFinal'][0])

['patient', 'board', 'catherine', 'mrn', 'euh_02058777', 'fin', '39093702276', 'age', '80', 'years', 'sex', 'female', 'dob', '5', '25', '1932', 'associated', 'diagnoses', 'none', 'author', 'sadeghi', 'yarandi', 'shadi', 'discharge', 'information', 'discharge', 'summary', 'information', 'date', 'of', 'admission', '10', '2', '2012', 'date', 'of', 'discharge', '10', '10', '2012', 'reason', 'for', 'admission', 'acute', 'decompensated', 'right', 'side', 'heart', 'failure', 'discharge', 'diagnosis', 'congestive', 'heart', 'failure', 'exacerbation', 'severely', 'depressed', 'rv', 'function', 'lvef', '55', 'atrial', 'fibrillation', 's', 'p', 'pacemaker', 'currently', 'paced', 'rate', 'controlled', 'copd', 'diabetes', 'mellituslast', 'hgba', 'c', '6', '2', 'history', 'of', 'coronary', 'artery', 'disease', 'chronic', 'kidney', 'disease', 'stage', 'iii', 'gout', 'results', 'review', 'radiology', 'results', 'echocardiogram', '8', '2012', '1', 'left', 'ventricular', 'ejection', 'fraction', 'is', '5

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['dcNoteFinal']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['Tag'],test_size=0.3)

In [ ]:
print(Corpus['text_final'].shape)

In [ ]:
Encoder = LabelEncoder()
Y = Encoder.fit_transform(Corpus['Tag'])

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
X = Tfidf_vect.transform(Corpus['text_final'])

In [ ]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
print(Tfidf_vect.vocabulary_)

{'patient': 3393, 'board': 619, 'catherine': 802, 'mrn': 2989, 'fin': 1722, 'age': 122, 'year': 4974, 'sex': 3980, 'female': 1706, 'dob': 1451, 'associated': 403, 'diagnosis': 1370, 'none': 3152, 'author': 452, 'sadeghi': 3861, 'yarandi': 4972, 'shadi': 3983, 'discharge': 1417, 'information': 2237, 'summary': 4281, 'date': 1264, 'admission': 92, 'reason': 3683, 'acute': 65, 'decompensated': 1292, 'right': 3809, 'side': 4016, 'heart': 1944, 'failure': 1686, 'congestive': 1072, 'exacerbation': 1643, 'severely': 3978, 'depress': 1345, 'rv': 3853, 'function': 1784, 'lvef': 2692, 'atrial': 426, 'fibrillation': 1714, 'pacemaker': 3351, 'currently': 1225, 'pace': 3350, 'rate': 3672, 'control': 1121, 'copd': 1139, 'diabetes': 1366, 'mellituslast': 2826, 'hgba': 2008, 'history': 2017, 'coronary': 1147, 'artery': 376, 'disease': 1423, 'chronic': 897, 'kidney': 2457, 'stage': 4167, 'iii': 2143, 'gout': 1850, 'result': 3781, 'review': 3793, 'radiology': 3661, 'echocardiogram': 1520, 'leave': 2552,

In [ ]:
print(Train_X_Tfidf.shape)

(105, 5000)


In [ ]:
print(Train_Y.shape)

(105,)


In [ ]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  56.52173913043478


In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  65.21739130434783


Cross Validation

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['Tag'],test_size=0.3)

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(Corpus['text_final'],Corpus['Tag'], test_size=0.3, random_state=0)

In [ ]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

X, y = datasets.load_iris(return_X_y=True)
X.shape, y.shape

((150, 4), (150,))

In [ ]:
print(X)
print(y)

In [ ]:
from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, X,Y, cv=5, n_splits=2)

TypeError: ignored

In [ ]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
import numpy
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset

# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X, Y):
  # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(X[train], Y[train], epochs=150, batch_size=10, verbose=0)
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

InvalidArgumentError: ignored

In [ ]:
import numpy as np
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=2)

for train, test in skf.split(X,Y):
    print("%s %s" % (train,test))

    clf = svm.SVC(kernel='linear', C=1)

    scores = cross_val_score(clf, train, test, cv=5)

[ 40  61  62  63  64  66  68  69  70  71  73  74  75  76  77  78  79  80
  82  83  84  86  87  89  90  91  92  94  95  97  98  99 100 103 104 105
 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123
 124 127 131 132 133 134 135 137 138 139 140 141 142 143 144 145 146 147
 148 149 150] [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  65  67  72  81  85  88  93  96 101 102 125 126
 128 129 130 136]


ValueError: ignored